In [644]:
import os
import pandas as pd

combined_df = []

for entry in os.scandir("example"):  
    if entry.is_file() and entry.name.endswith(".csv"):
        df = pd.read_csv(entry.path)
        combined_df.append(df)

combined_df = pd.concat(combined_df, ignore_index=True)

print(combined_df)

     nr.   xpix.  ypix.    xmm   ymm  powierzchniamm  dlugoscmm  szerokoscmm  \
0      1   586.5  432.0   58.7  43.2            0.40        0.7          0.7   
1      2   451.0  639.0   45.1  63.9            0.39        0.7          0.7   
2      3   614.5  750.5   61.5  75.0            1.10        1.4          0.8   
3      4   881.5  159.0   88.2  15.9            0.58        1.1          0.6   
4      5  1117.0   85.5  111.7   8.6            0.58        1.1          0.3   
..   ...     ...    ...    ...   ...             ...        ...          ...   
474   43   191.5   59.0   19.2   5.9            0.81        1.4          0.7   
475   44   488.0  658.5   48.8  65.9            1.42        1.6          1.2   
476   45   381.5  558.0   38.1  55.8            2.03        4.0          0.7   
477   46  1122.5  738.0  112.2  73.8            1.30        1.7          0.9   
478   47  1266.5  333.5  126.7  33.4            1.31        2.7          0.6   

        kat   obwodmm  ...  sredn._mart

In [645]:
import pandas as pd

# df = pd.read_csv("example/a1_raport.csv")
df = combined_df

In [646]:
df.head()

,nr.,xpix.,ypix.,xmm,ymm,powierzchniamm,dlugoscmm,szerokoscmm,kat,obwodmm,...,sredn._martinamm,sredn._sitowamm,srednia_jaskrawosc,r,g,b,y,liczenie_obiektow_w,udzial_punktow,pole_obrazu_mm2
0,1,586.5,432.0,58.7,43.2,0.40,0.7,0.7,121.39,2.414214,...,0.7,0.713650,135.4,97.6,148.1,169.3,135.4,1,7.3,7.0
1,2,451.0,639.0,45.1,63.9,0.39,0.7,0.7,0.00,2.331371,...,0.7,0.704673,122.4,85.8,134.2,157.9,122.4,1,6.9,7.3
2,3,614.5,750.5,61.5,75.0,1.10,1.4,0.8,177.54,4.179899,...,1.4,1.183454,154.1,117.7,165.7,190.0,154.1,1,11.2,11.6
3,4,881.5,159.0,88.2,15.9,0.58,1.1,0.6,45.72,2.897056,...,1.1,0.859348,137.1,97.0,151.0,170.5,137.1,1,8.7,8.1
4,5,1117.0,85.5,111.7,8.6,0.58,1.1,0.3,160.67,2.931371,...,1.1,0.859348,106.3,70.4,116.3,149.1,106.3,1,8.9,8.1


In [647]:
# Scaling
df['dlugoscmm'] = df['dlugoscmm']/1.49
df['szerokoscmm'] = df['szerokoscmm']/1.49
df['powierzchniamm'] = df['powierzchniamm']/2.235

In [648]:
# Set proble volume in mL
PROBE_VOLUME_ML = 6
# PROBE_VOLUME_ML_SCALER = 1000 / PROBE_VOLUME_ML

In [649]:
df.shape

(479, 23)

In [650]:
df = df[df['dlugoscmm'] >= 0.2]
df = df[df['szerokoscmm'] <= 1.5]

In [651]:
df.shape

(477, 23)

In [652]:
df['Pw'] = 3.14*(df['szerokoscmm']/2)**2+df['szerokoscmm']*(df['dlugoscmm']-df['szerokoscmm'])

In [653]:
df['R'] = df['Pw'] / df['powierzchniamm']

In [654]:
df['Dk'] = df['dlugoscmm']
df['Sk'] = df['szerokoscmm']
df['D/S'] = (df['Dk'] / df['Sk']).astype('float64')

In [655]:
bacteria_types = []
Dks = []
Sks = []
for idx,row  in df.iterrows():
    if row['R'] > 1.2:
        Dks.append(row['szerokoscmm'])
        Sks.append(row['Pw']*0.8)
        bacteria_types.append("Krzywe")
    else:
        Dks.append(row['dlugoscmm'])
        Sks.append(row['szerokoscmm'])
        if round(row['D/S'],15) > 1.5:
            bacteria_types.append("Pałeczki")
        else:
            bacteria_types.append("Ziarniaki")
df['bacteria_type'] = bacteria_types
df['Dk'] = Dks
df['Sk'] = Sks

In [656]:
df['Ob'] =(3.14*(df['Sk']**3)/6)+(3.14*((df['Sk']**2)/4)*(df['Dk']-df['Sk']))

In [657]:
df = df[df['Ob']>0]

In [658]:
df['biomasa'] =((104.5*(df['Ob']**0.59))*0.86)/1000000*PROBE_VOLUME_ML_SCALER

In [659]:
df['bialko'] = 104.5 * (df['Ob']**0.59)

In [660]:
df['wegiel'] = 0.86 * df['bialko']

In [661]:
df['bacteria_type'].value_counts()

bacteria_type
Pałeczki     224
Ziarniaki    165
Krzywe        86
Name: count, dtype: int64

In [671]:
grouped = df.groupby("bacteria_type")
result_count = grouped.size().reset_index(name="count")

In [672]:
grouped = df.groupby("bacteria_type")
result = grouped["Ob"].mean().reset_index()

In [673]:
result_count

,bacteria_type,count
0,Krzywe,86
1,Pałeczki,224
2,Ziarniaki,165


In [675]:
result['bialko'] = 104.5 * (result['Ob']**0.59)
result['wegiel'] = 0.86 * result['bialko']
result['count_in_1_ml']=((result_count['count']*48097.39)/10)/(PROBE_VOLUME_ML)
result['biomasa'] =(((104.5*result['Ob']**0.59)*0.86)*result['count_in_1_ml'])/1000000

In [676]:
result

,bacteria_type,Ob,bialko,wegiel,count_in_1_ml,biomasa
0,Krzywe,0.086847,24.716134,21.255875,68939.592333,1.465371
1,Pałeczki,0.198839,40.293205,34.652156,179563.589333,6.222266
2,Ziarniaki,0.130435,31.419407,27.020690,132267.822500,3.573968


In [ ]:
bins = [0, 0.1, 0.2, 0.5, 1.0, float("inf")]
labels = ["<=0.1", "0.1–0.2", "0.2–0.5", "0.5–1.0", ">1.0"]

df["Ob_bucket"] = pd.cut(df["Ob"], bins=bins, labels=labels, right=True)

grouped = df.groupby(["bacteria_type", "Ob_bucket"])
result = grouped.size().reset_index(name="count")

C:\Users\Kuba\AppData\Local\Temp\ipykernel_19784\3292400346.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(["bacteria_type", "Ob_bucket"])


In [ ]:
df[df['bacteria_type']=='Pałeczki']

,nr.,xpix.,ypix.,xmm,ymm,powierzchniamm,dlugoscmm,szerokoscmm,kat,obwodmm,...,R,Dk,Sk,D/S,bacteria_type,Ob,biomasa,bialko,wegiel,Ob_bucket
2,3,614.5,750.5,61.5,75.0,0.492170,0.939597,0.536913,177.54,4.179899,...,0.899085,0.939597,0.536913,1.750000,Pałeczki,0.172127,0.005304,37.005553,31.824776,0.1–0.2
3,4,881.5,159.0,88.2,15.9,0.259508,0.738255,0.402685,45.72,2.897056,...,1.011224,0.738255,0.402685,1.833333,Pałeczki,0.076888,0.003297,23.002299,19.781977,<=0.1
4,5,1117.0,85.5,111.7,8.6,0.259508,0.738255,0.201342,160.67,2.931371,...,0.539198,0.738255,0.201342,3.666667,Pałeczki,0.021358,0.001548,10.803209,9.290760,<=0.1
8,9,127.5,613.0,12.8,61.3,0.425056,0.939597,0.604027,171.30,4.062742,...,1.150671,0.939597,0.604027,1.555556,Pałeczki,0.211441,0.005989,41.780876,35.931553,0.2–0.5
9,10,975.0,499.5,97.5,50.0,0.550336,1.208054,0.201342,54.43,4.487006,...,0.426134,1.208054,0.201342,6.000000,Pałeczki,0.036308,0.002118,14.774665,12.706212,<=0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,39,6.5,157.0,0.7,15.7,0.196868,0.872483,0.067114,13.37,3.072792,...,0.292518,0.872483,0.067114,13.000000,Pałeczki,0.003006,0.000487,3.397195,2.921588,<=0.1
471,40,553.0,951.0,55.3,95.1,0.263982,0.805369,0.067114,37.40,3.131371,...,0.201086,0.805369,0.067114,12.000000,Pałeczki,0.002769,0.000464,3.236296,2.783215,<=0.1
474,43,191.5,59.0,19.2,5.9,0.362416,0.939597,0.469799,110.01,3.755635,...,1.087062,0.939597,0.469799,2.000000,Pałeczki,0.135661,0.004609,32.156146,27.654285,0.1–0.2
477,46,1122.5,738.0,112.2,73.8,0.581655,1.140940,0.604027,8.47,4.804163,...,1.049961,1.140940,0.604027,1.888889,Pałeczki,0.269106,0.006904,48.169418,41.425699,0.2–0.5


In [ ]:
df[(df['bacteria_type'] == 'Pałeczki') & (df['nr.'] == 35)]


,nr.,xpix.,ypix.,xmm,ymm,powierzchniamm,dlugoscmm,szerokoscmm,kat,obwodmm,...,R,Dk,Sk,D/S,bacteria_type,Ob,biomasa,bialko,wegiel,Ob_bucket
34,35,361.5,86.0,36.1,8.6,0.729306,1.409396,0.469799,174.47,5.238478,...,0.842827,1.409396,0.469799,3.000000,Pałeczki,0.217057,0.006082,42.432124,36.491627,0.2–0.5
134,35,1190.0,333.5,119.0,33.4,6.572707,7.046980,1.140940,2.33,25.162236,...,1.180686,7.046980,1.140940,6.176471,Pałeczki,6.812456,0.046464,324.164517,278.781485,>1.0
172,35,427.5,952.5,42.8,95.2,0.290828,0.805369,0.335570,87.47,3.672792,...,0.846025,0.805369,0.335570,2.400000,Pałeczki,0.061304,0.002885,20.125012,17.307511,<=0.1
266,35,727.5,144.0,72.8,14.4,0.666667,1.140940,0.604027,9.15,4.721320,...,0.916074,1.140940,0.604027,1.888889,Pałeczki,0.269106,0.006904,48.169418,41.425699,0.2–0.5
318,35,461.0,421.5,46.1,42.2,0.174497,0.536913,0.268456,21.51,2.331371,...,0.737223,0.536913,0.268456,2.000000,Pałeczki,0.025313,0.001712,11.942263,10.270346,<=0.1
418,35,416.5,435.0,41.7,43.5,0.371365,1.073826,0.201342,53.80,3.887006,...,0.558725,1.073826,0.201342,5.333333,Pałeczki,0.032036,0.001967,13.722923,11.801714,<=0.1
466,35,1269.5,192.5,127.0,19.2,0.478747,1.208054,0.067114,27.20,4.321320,...,0.167330,1.208054,0.067114,18.000000,Pałeczki,0.004192,0.000593,4.134004,3.555244,<=0.1


In [ ]:
df[df['bacteria_type']=='Krzywe']

,nr.,xpix.,ypix.,xmm,ymm,powierzchniamm,dlugoscmm,szerokoscmm,kat,obwodmm,...,R,Dk,Sk,D/S,bacteria_type,Ob,biomasa,bialko,wegiel,Ob_bucket
6,7,781.0,57.0,78.1,5.7,0.340045,1.006711,0.536913,30.11,3.804163,...,1.407277,0.536913,0.382830,1.875000,Krzywe,0.047090,0.002469,17.224330,14.812923,<=0.1
29,30,304.0,319.5,30.4,32.0,0.304251,0.738255,0.604027,39.96,3.131371,...,1.207832,0.604027,0.293987,1.222222,Krzywe,0.034332,0.002049,14.294889,12.293605,<=0.1
35,36,241.0,793.5,24.1,79.4,0.299776,0.738255,0.604027,142.89,3.072792,...,1.225859,0.604027,0.293987,1.222222,Krzywe,0.034332,0.002049,14.294889,12.293605,<=0.1
36,37,485.5,695.5,48.6,69.5,0.451902,1.140940,0.536913,68.55,4.379899,...,1.218420,0.536913,0.440485,2.125000,Krzywe,0.059414,0.002832,19.756562,16.990643,<=0.1
46,47,214.5,467.5,21.5,46.8,0.442953,1.073826,0.604027,40.58,5.218377,...,1.287218,0.604027,0.456142,1.777778,Krzywe,0.073822,0.003219,22.456763,19.312817,<=0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,17,843.0,401.0,84.3,40.1,0.447427,1.073826,0.604027,101.79,4.179899,...,1.274346,0.604027,0.456142,1.777778,Krzywe,0.073822,0.003219,22.456763,19.312817,<=0.1
449,18,100.5,643.5,10.1,64.4,0.425056,1.006711,0.604027,33.23,4.038478,...,1.246044,0.604027,0.423711,1.666667,Krzywe,0.065222,0.002992,20.874100,17.951726,<=0.1
451,20,680.0,697.0,68.0,69.7,0.581655,1.140940,0.738255,75.98,4.828427,...,1.246657,0.738255,0.580100,1.545455,Krzywe,0.143940,0.004773,33.299998,28.637999,0.1–0.2
469,38,443.5,48.0,44.4,4.8,0.469799,1.006711,0.671141,51.40,4.652691,...,1.232023,0.671141,0.463042,1.500000,Krzywe,0.086982,0.003546,24.738807,21.275374,<=0.1


In [ ]:
result

,bacteria_type,Ob_bucket,count
0,Krzywe,<=0.1,67
1,Krzywe,0.1–0.2,11
2,Krzywe,0.2–0.5,6
3,Krzywe,0.5–1.0,2
4,Krzywe,>1.0,0
5,Pałeczki,<=0.1,112
6,Pałeczki,0.1–0.2,52
7,Pałeczki,0.2–0.5,43
8,Pałeczki,0.5–1.0,12
9,Pałeczki,>1.0,5


In [ ]:
import numpy as np
total_bacteria_count = np.sum(result['count'])
total_bacteria_count_1_ml = ((np.sum(result['count'])*48097.39)/10)/(PROBE_VOLUME_ML)

In [ ]:
result['count_in_1_ml']=((result['count']*48097.39)/10)/(PROBE_VOLUME_ML)

In [ ]:

result['bio_diversity'] =((result['count_in_1_ml']+1)/total_bacteria_count_1_ml)*np.log10((result['count_in_1_ml']+1)/total_bacteria_count_1_ml)

In [ ]:
result['bio_diversity']

0    -0.119983
1    -0.037873
2    -0.023985
3    -0.010008
4    -0.000015
5    -0.147953
6    -0.105172
7    -0.094441
8    -0.040361
9    -0.020822
10   -0.141423
11   -0.090497
12   -0.065509
13   -0.010008
14   -0.005641
Name: bio_diversity, dtype: float64

In [ ]:
result

,bacteria_type,Ob_bucket,count,count_in_1_ml,bio_diversity
0,Krzywe,<=0.1,67,53708.752167,-0.119983
1,Krzywe,0.1–0.2,11,8817.854833,-0.037873
2,Krzywe,0.2–0.5,6,4809.739000,-0.023985
3,Krzywe,0.5–1.0,2,1603.246333,-0.010008
4,Krzywe,>1.0,0,0.000000,-0.000015
5,Pałeczki,<=0.1,112,89781.794667,-0.147953
6,Pałeczki,0.1–0.2,52,41684.404667,-0.105172
7,Pałeczki,0.2–0.5,43,34469.796167,-0.094441
8,Pałeczki,0.5–1.0,12,9619.478000,-0.040361
9,Pałeczki,>1.0,5,4008.115833,-0.020822


In [ ]:
shannon_index = np.sum(result['bio_diversity'])*-1
# shannon_index_1_L = np.sum(result['bio_diversity_1_L'])*-1
print(f"Shannon index: {shannon_index}")

Shannon index: 0.9136913415400122
